In [1]:
# load required modules
import ee
ee.Initialize()

In [2]:
# Import map
import geemap
Map = geemap.Map()

## Create composites using the yearly MODIS phenology product 

In [5]:
# function for extracting quality bits
def getQABits(image, start, end, mascara):
    # Compute the bits we need to extract.
    pattern = 0
    for i in range(start,end+1):
        pattern += 2**i
    # Return a single band image of the extracted QA bits, giving the     band a new name.
    return image.select([0], [mascara]).bitwiseAnd(pattern).rightShift(start)
    
# function for filtering image based on quality bits
def mask_pixelsall(bstart,bend):
    def maskPixels(image0):
        tmp = image0.select('QA_Detailed_1')
        quality = getQABits(tmp, bstart, bend, 'QA_Detailed_1')
        # Create a mask that filters out undesired areas
        mask = quality.eq(0)
        return image0.updateMask(mask) 
    return(maskPixels)

# function for masking areas where there have not land use changes. This layer was created using the ESA cci map
def mask_image(image):
    image = image.updateMask(MakMarco.eq(1))
    return image

# convert feature collection into feature collection with no geometry (easier to save)
def convert(feature):
    res = ee.Feature(None,feature.toDictionary())
    return(res)

In [6]:
# ---- load datasets
# load MODIS phenology product
modis_phenoprod = ee.ImageCollection('MODIS/006/MCD12Q2')
# load mask of unchanged forest pixels
MakMarco = ee.Image("users/marcogirardello/phenoutils/mask_unchanged_500m")



In [7]:
year = 2001

In [8]:
from IPython.display import JSON

In [32]:
#JSON(pheno_tmp.getInfo())

In [9]:
# filter for year of interest
pheno_tmp = modis_phenoprod.filter(ee.Filter.date(str(year)+'-01-01', str(year+1)+'-01-01'))

numc = pheno_tmp.select("NumCycles").first()

# ------- Greenup
GUP = pheno_tmp.select(['Greenup_1','QA_Detailed_1'])
# only take pixels above a certain threshold level
GUP1 = GUP.map(mask_pixelsall(bstart=0,bend=1)).select('Greenup_1').first()
# filter using mask where pixels have been stable
GUP2 = GUP1.updateMask(MakMarco.eq(1))

In [14]:
ovqual = pheno_tmp.select("QA_Overall_1").first()

In [15]:
vegetationPeakVis = {
  'min': 11400,
  'max': 11868,
  'palette': ['0f17ff', 'b11406', 'f1ff23'],
}


In [16]:
MGUP = pheno_tmp.select(['MidGreenup_1','QA_Detailed_1'])
# only take pixels above a certain threshold level
MGUP1 = MGUP.map(mask_pixelsall(bstart=2,bend=3)).select('MidGreenup_1').first()

In [17]:
Map.addLayer(GUP.first().select('Greenup_1'),vegetationPeakVis,'Greenup_nofilt')

In [15]:
Map.addLayer(GUP1,vegetationPeakVis,'Greenup_filt')

In [45]:
Map.addLayer(GUP1.updateMask(ovqual.eq(0)),vegetationPeakVis,'Greenup_filt 1')

In [25]:
Map.addLayer(numc,'','number of cycles')

In [66]:
Map.addLayer(MGUP1.updateMask(ovqual.eq(0)),vegetationPeakVis,'MGUP1')

In [22]:
tmp = MGUP1.updateMask(ovqual.eq(0)).updateMask(MakMarco.eq(1))

In [19]:
tmp1 = tmp.reduceResolution(ee.Reducer.count(),maxPixels=300,bestEffort = False).reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1)

In [23]:
tmp2 = tmp.reduceResolution(ee.Reducer.stdDev(),maxPixels=300,bestEffort = False).reproject(ee.Projection('EPSG:4326').scale(0.05, 0.05)).updateMask(1)

In [43]:
Map.addLayer(ovqual1,'','ovqual')

In [21]:
Map.addLayer(tmp1,{'min':1,'max':150,'palette':['blue','green','yellow','red']},'count')

In [26]:
Map.addLayer(tmp2,{'min':1,'max':10,'palette':['blue','green','yellow','red']},'diversity')

In [20]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [11]:
# years
years = list([2016])

for year in years:
    # filter for year of interest
    pheno_tmp = modis_phenoprod.filter(ee.Filter.date(str(year)+'-01-01', str(year+1)+'-01-01'))
    # ------- Greenup
    GUP = pheno_tmp.select(['Greenup_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    GUP1 = GUP.map(mask_pixelsall(bstart=0,bend=1)).select('Greenup_1').first()
    # filter using mask where pixels have been stable
    GUP2 = GUP1.updateMask(MakMarco.eq(1))
    #Export image to google drive
    task = ee.batch.Export.image.toDrive(image =  GUP2,description ='GUP_'+str(year),folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
    task.start()
    
    # ------- MidGreenup
    MGUP = pheno_tmp.select(['MidGreenup_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    MGUP1 = MGUP.map(mask_pixelsall(bstart=2,bend=3)).select('MidGreenup_1').first()
    # filter using mask where pixels have been stable
    MGUP2 = MGUP1.updateMask(MakMarco.eq(1))
    #Export image to google drive
    task = ee.batch.Export.image.toDrive(image =  MGUP2,description ='MGUP_'+str(year),folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
    task.start()
    
    # ------- Maturity
    MAT = pheno_tmp.select(['Maturity_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    MAT1 = MAT.map(mask_pixelsall(bstart=4,bend=5)).select('Maturity_1').first()
    # filter using mask where pixels have been stable
    MAT2 = MAT1.updateMask(MakMarco.eq(1))
    #Export image to google drive
    task = ee.batch.Export.image.toDrive(image =  MAT2,description ='MAT_'+str(year),folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
    task.start()
    
    # ------- Peak
    Peak = pheno_tmp.select(['Peak_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    Peak1 = Peak.map(mask_pixelsall(bstart=6,bend=7)).select('Peak_1').first()
    # filter using mask where pixels have been stable
    Peak2 = Peak1.updateMask(MakMarco.eq(1))
    #Export image to google drive
    task = ee.batch.Export.image.toDrive(image =  Peak2,description ='Peak_'+str(year),folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
    task.start()
    
    # ------- Senescence
    SEN = pheno_tmp.select(['Senescence_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    SEN1 = SEN.map(mask_pixelsall(bstart=8,bend=9)).select('Senescence_1').first()
    # filter using mask where pixels have been stable
    SEN2 = SEN1.updateMask(MakMarco.eq(1))
    #Export image to google drive
    task = ee.batch.Export.image.toDrive(image =  SEN2,description ='SEN_'+str(year),folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
    task.start()
    
    # ------- MidGreendown
    MGDO = pheno_tmp.select(['MidGreendown_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    MGDO1 = MGDO.map(mask_pixelsall(bstart=10,bend=11)).select('MidGreendown_1').first()
    # filter using mask where pixels have been stable
    MGDO2 = MGDO1.updateMask(MakMarco.eq(1))
    #Export image to google drive
    task = ee.batch.Export.image.toDrive(image =  MGDO2,description ='MGDO_'+str(year),folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
    task.start()

    # ------- Dormancy
    Dormancy = pheno_tmp.select(['Dormancy_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    Dormancy1 = Dormancy.map(mask_pixelsall(bstart=12,bend=13)).select('Dormancy_1').first()
    # filter using mask where pixels have been stable
    Dormancy2 = Dormancy1.updateMask(MakMarco.eq(1))
    #Export image to google drive
    task = ee.batch.Export.image.toDrive(image =  Dormancy2,description ='Dormancy_'+str(year),folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
    task.start()



In [51]:
# lists where images will be appended
# Greenup
GUPl = ee.List([])
# Peak
Peakl = ee.List([])
# Dormancy
Dormancyl = ee.List([])

# years
years = list(range(2001, 2016+1))

for year in years:
    # filter for year of interest
    pheno_tmp = modis_phenoprod.filter(ee.Filter.date(str(year)+'-01-01', str(year+1)+'-01-01'))
    # ------- Greenup
    GUP = pheno_tmp.select(['Greenup_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    GUP1 = GUP.map(mask_pixelsall(bstart=0,bend=1)).select('Greenup_1').first()
    # filter using mask where pixels have been stable
    GUP2 = GUP1.updateMask(MakMarco.eq(1))
    # append to list
    GUPl = GUPl.add(GUP2)
    # ------- Peak
    # subset Peak for given year
    Peak = pheno_tmp.select(['Peak_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    Peak1 = Peak.map(mask_pixelsall(bstart=6,bend=7)).select('Peak_1').first()
    # filter using mask where pixels have been stable
    Peak2 = Peak1.updateMask(MakMarco.eq(1))
    # append to list
    Peakl = Peakl.add(Peak2)
    # ------- Dormancy
    Dormancy = pheno_tmp.select(['Dormancy_1','QA_Detailed_1'])
    # only take pixels above a certain threshold level
    Dormancy1 = Dormancy.map(mask_pixelsall(bstart=12,bend=13)).select('Dormancy_1').first()
    # filter using mask where pixels have been stable
    Dormancy2 = Dormancy1.updateMask(MakMarco.eq(1))
    # append to list
    Dormancyl = Dormancyl.add(Dormancy2)

In [52]:
# convert lists into image collection
GUPic = ee.ImageCollection(GUPl)
Peakic = ee.ImageCollection(Peakl)
Dormancyic = ee.ImageCollection(Dormancyl)

In [53]:
# calculate the median of the collections across years
GUPicm = GUPic.median()
Peakicm = Peakic.median()
Dormancyicm = Dormancyic.median()

In [ ]:
#Export collection medians to google drive
# GUP
task = ee.batch.Export.image.toDrive(image = GUPicm,description ='GUP',folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
task.start()
# Peak
task = ee.batch.Export.image.toDrive(image = Peakicm,description ='GUP',folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
task.start()
# Dormancy
task = ee.batch.Export.image.toDrive(image = Dormancyicm,description ='GUP',folder="phenology",scale = 463.3127165275,
                                        fileFormat='GeoTIFF',skipEmptyTiles = True, crs ='EPSG:4326',maxPixels=1e13)
task.start()